In [17]:
import yfinance as yf
import datetime as dt
import numpy as np
from scipy.stats import norm
import pandas as pd
import matplotlib.pyplot as plt

In [4]:
start = "2024-05-01"
end = dt.datetime.now()
stock = yf.download("AAPL", start, end)
stock_adj = stock["Close"]

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed


In [5]:
r = yf.download("^TNX", start, end)["Close"].iloc[-1].item()/100
#q = yf.Ticker("AAPL").info.get("dividendYield")
#sigma = stock_adj.pct_change().std()*np.sqrt(252)


[*********************100%***********************]  1 of 1 completed


In [29]:
class Option:
    def __init__(self, ticker, K, T, type, T0):
        self.ticker = ticker
        self.K = K
        self.T = T
        self.type = type
        self.T0 = T0
        self.Moneyness()
        self.BS()
        self.Payoff()
        self.Tracer

    def __repr__(self):
        return self.type + " Option on " + self.ticker + " at Maturity " + str(self.T) + " year " + "and strike price " + str(self.K)

    def Moneyness(self):
        if self.type == "call":
            if stock_adj.loc[self.T0].item() > self.K:
                return "ITM"
            if stock_adj.loc[self.T0].item() == self.K:
                return "ATM"
            if stock_adj.loc[self.T0].item() < self.K:
                return "OTM"
        if self.type == "put":
            if stock_adj.loc[self.T0].item() > self.K:
                return "OTM"
            if stock_adj.loc[self.T0].item() == self.K:
                return "ATM"
            if stock_adj.loc[self.T0].item() < self.K:
                return "ITM"

    def Payoff(self):
        S = stock_adj.iloc[-1].item()
        if self.type == "call":
            return max(S - self.K, 0)
        if self.type == "put":
            return max(self.K - S, 0)
               

    def BS(self):
        q = yf.Ticker(self.ticker).info.get("dividendYield")/100
        sigma = yf.download(self.ticker, start, end)["Close"].pct_change().std()*np.sqrt(252)
        S0 = stock_adj.loc[self.T0].item()
        d1 = (np.log(S0/self.K) + (r - q + (1/2)*sigma**2))*self.T/(sigma*np.sqrt(self.T))
        d2 = d1 - sigma*np.sqrt(self.T)
        if self.type == "call":
            return (S0*np.exp(-q*self.T)*norm.cdf(d1) - self.K*np.exp(-r*self.T)*norm.cdf(d2))[0]
        if self.type == "put":
            return (self.K*np.exp(-r*self.T)*norm.cdf(-d2) - S0*np.exp(-q*self.T)*norm.cdf(-d1))[0]
            
    def Tracer(self):
        C = self.BS()
        A = np.arange(0,3,0.01)
        if self.type == "call":
            G = [ (i*self.K - self.K - C if i > 1 else -C) for i in A]
        if self.type == "put":
            G = [ (self.K -i*self.K - C if i < 1 else -C) for i in A]
        plt.title('P/L Option')
        plt.plot(A,G)
        plt.xlabel('Percentage Increase')
        plt.ylabel('P/L')
        plt.grid(True)
        plt.show()


            

In [30]:
test = Option("AAPL", 209, 1, "call", "2025-03-13")
test


[*********************100%***********************]  1 of 1 completed


call Option on AAPL at Maturity 1 year and strike price 209